# Applying the Method Of Manufactured Solutions to SWIRL

## Introduction

The method of manufactured solutions (MMS) is a general procedure for constructing an analytical solution and determining the accuracy via convergence rate. As a result, MMS serves as a means of code verification. In this work, SWIRL, a code used to calculate the radial modes within an infinitely long duct, is validated using MMS. SWIRL has two components that require numerical techniques and approximation. First, SWIRL accepts a given mean flow and uses numerical integration to obtain the speed of sound.  Secondly, SWIRL constructs a linear matrix equation using the Linearized Euler Equations (LEE) as a function of radius. This work aims to use the Method of Manufactured solutions to reassure that these two steps lead to accurate results.

In [1]:
# Set Up Libraries
%run set-up.ipynb
%run symbolic-variables.ipynb
%run mms-mean-flow.ipynb
# import functions
%run SourceSubstitutionFunction.ipynb

### Defining Linearized Euler Equation (LEE) Source Terms

Initially the source terms were defined without mention of the indices of the matrices they make up. In other words, there was no fore sight on the fact that these source terms are sums 
of the elements within A,B, and X. To investigate the source terms in greater detail, the FORTRAN code that calls the source terms will output the terms within the source term and then sum them, instead of just their sum.

#### Eigenvalue Decomposition


$$ [A]{x} = \lambda [B] {x} $$

which can be rearranged as,

$$ [A]{x} - \lambda [B] {x} = 0$$

Here, $x$ is an eigenvector composed of the perturbation variables, $v_r,v_{\theta},v_x,p$ and $\lambda$ is the associated eigenvalue, (Note: $\lambda = -i \bar{\gamma}$)

Writing this out we obtain ....

Linear System of Equations:
\begin{equation}
-
i \left(
\frac{k}{A} - \frac{m}{r} M_{\theta}
\right)
v_r 
-
\frac{2}{r} M_{\theta} v_{\theta} 
+
\frac{dp}{dr} 
+
\frac{(\kappa - 1)}{r} M_{\theta}^2 p
-
\lambda M_x v_r =S_1
\end{equation}

Using matrix notation,

\begin{equation}
A_{11}
x_1 
-
A_{12} x_2 
+
A_{14} x_4
-
\lambda B_{11} x_1 = S_1
\end{equation}

But $A_{14}$ and $A_{41}$ in Kousen's paper only has the derivative operator. Since I am currntly writing the matrix out term by term and not doing the matrix math to obtain the symbolic expressions, I will define $A_{14}$ with $dp/dr$ and $A_{41}$ with $dv_r/dr$
Similarly,
\begin{align}
A_{21} x_1 &-
A_{22} x_2 +
A_{24} x_4 &-
\lambda B_{22} x_2 &= S_2 \\
A_{31} x_1 &-
A_{33} x_3 &-
\lambda (B_{33} x_3 + B_{34} x_4) &= S_3\\
A_{41} x_1 &+
A_{42} x_2 +
A_{44} x_4 &- 
\lambda (B_{33} x_3 + B_{44} x_4) &= S_4
\end{align}
Now we can begin looking at the source terms, term by term. They each should also converge at a known rate

In [2]:
%run SourceTermSystemOfEquations.ipynb

Now lets get these source terms by using the matrix form of the equations and see if they're
equal

In [3]:
%run SourceTermMatrixEquation.ipynb
# Checking if the matrix expressions equal the linear system of equations
print(S[0].equals(SS[0]))
print(S[1].equals(SS[1]))
print(S[2].equals(SS[2]))
print(S[3].equals(SS[3]))

True
True
True
True


## Generating a block diagonal matrix 

In [4]:
#now that we set the matricies up lets get our individual terms 
A_times_x        = sp.Matrix(sp.zeros(len(S),len(S)))
lambda_B_times_x = sp.Matrix(sp.zeros(len(S),len(S)))

for j in range(len(S)):
    for i in range(len(S)):
        A_times_x[i,j]        = [AA[i,j]*XX[j]]
        lambda_B_times_x[i,j] = Lambda*BB[i,j]*XX[j]

In [5]:
for i in range(len(S)):
    match =(S[i].equals( \
                  A_times_x[i,0] + \
                  A_times_x[i,1] + \
                  A_times_x[i,2] + \
                  A_times_x[i,3] -\
                 (lambda_B_times_x[i,0] + \
                  lambda_B_times_x[i,1] + \
                  lambda_B_times_x[i,2] + \
                  lambda_B_times_x[i,3])))
    print(match)
# notes on differences between system of equations and Ax-lambda Bx    
# multiplying (1/p) and not (one/p) by dp_dr made  S_1 and SS[1] the same
# multiplied i gamma p term by "one" in S_3
# multiplied by 1/v_r by d_v_dr and i v_x gamma terM by one

True
True
True
True


In [6]:
for i in range(len(S)):
    S[i] = SourceSubstitution(S[i], \
                       A_analytic         , \
                       M_t_analytic   , \
                       M_x_analytical     , \
                       v_r_analytical     , \
                       v_t_analytical     , \
                       v_x_analytical     , \
                       p_analytical       , \
                       dp_dr_analytical   , \
                       dv_r_dr_analytical , \
                       dM_x_dr_analytical , \
                       dM_t_dr_analytical , \
                       )

for i in range(len(A_times_x[:,0])):
    for j in range(len(A_times_x[0,:])):
        A_times_x[i,j] = SourceSubstitution(\
                       A_times_x[i,j], \
                       A_analytic         , \
                       M_t_analytic   , \
                       M_x_analytical     , \
                       v_r_analytical     , \
                       v_t_analytical     , \
                       v_x_analytical     , \
                       p_analytical       , \
                       dp_dr_analytical   , \
                       dv_r_dr_analytical , \
                       dM_x_dr_analytical , \
                       dM_t_dr_analytical , \
                                      ) 
        lambda_B_times_x[i,j] = SourceSubstitution(\
                       lambda_B_times_x[i,j], \
                       A_analytic         , \
                       M_t_analytic   , \
                       M_x_analytical     , \
                       v_r_analytical     , \
                       v_t_analytical     , \
                       v_x_analytical     , \
                       p_analytical       , \
                       dp_dr_analytical   , \
                       dv_r_dr_analytical , \
                       dM_x_dr_analytical , \
                       dM_t_dr_analytical , \
                                         )

In [7]:
%run create-fortran-file-sound-speed.ipynb

In [8]:
%run create-fortran-file-perturbation-variables.ipynb

In [9]:
S_1 = S[0]
S_2 = S[1]
S_3 = S[2]
S_4 = S[3]
%run create-fortran-file-LEE-source-terms.ipynb
%run create-fortran-file-LEE-source-terms-components.ipynb